In [20]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from prettytable import PrettyTable
from termcolor import cprint
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from plotly.offline import iplot
import plotly.graph_objs as go
import matplotlib.pyplot as plt

In [21]:
df=pd.read_csv("D:\Solytics\Datasets\scores.csv")

In [5]:
##%% FUNCTION FOR STEPWISE ELIMINATION

def stepwise(df):
    df=df.fillna(df.mean())
    le = LabelEncoder()
    cat_cols=list(df.select_dtypes('object').columns)
    for fet in cat_cols:
        #le.fit(df[fet])
        df[fet] = le.fit_transform(df[fet].astype(str))
    target=input('ENTER THE TARGET VARIABLE : ')
    Target=df[target]
    X=df.drop(columns=target)
    from mlxtend.feature_selection import SequentialFeatureSelector as SFS
    from sklearn.linear_model import LinearRegression
    fwd=int(input("Forward features to enter :"))
    bwd=int(input("Backward features to enter :"))
    if ((fwd!=0 and bwd!=0) and (fwd<bwd) and (fwd<=len(X))):
        sffs = SFS(LinearRegression(),
         k_features=(fwd,bwd),
         forward=True,
         floating=True,
         cv=0)
        sffs.fit(X, Target)
        print(sffs.k_feature_names_)
    else:
        print("Invalid inputs")

In [7]:
stepwise(df)

ENTER THE TARGET VARIABLE : BAD
Forward features to enter :1
Backward features to enter :1
Invalid inputs


In [53]:
#$$trial

import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 2

def IV(df):
    def mono_bin(Y, X, n = max_bin):

        df1 = pd.DataFrame({"X": X, "Y": Y})
        justmiss = df1[['X','Y']][df1.X.isnull()]
        notmiss = df1[['X','Y']][df1.X.notnull()]
        r = 0
        while np.abs(r) < 1:
            try:
                d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
                d2 = d1.groupby('Bucket', as_index=True)
                r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
                n = n - 1 
            except Exception as e:
                n = n - 1

        if len(d2) == 1:
            n = force_bin         
            bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
            if len(np.unique(bins)) == 2:
                bins = np.insert(bins, 0, 1)
                bins[1] = bins[1]-(bins[1]/2)
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
            d2 = d1.groupby('Bucket', as_index=True)

        d3 = pd.DataFrame({},index=[])
        d3["MIN_VALUE"] = d2.min().X
        d3["MAX_VALUE"] = d2.max().X
        d3["COUNT"] = d2.count().Y
        d3["EVENT"] = d2.sum().Y
        d3["NONEVENT"] = d2.count().Y - d2.sum().Y
        d3=d3.reset_index(drop=True)

        if len(justmiss.index) > 0:
            d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
            d4["MAX_VALUE"] = np.nan
            d4["COUNT"] = justmiss.count().Y
            d4["EVENT"] = justmiss.sum().Y
            d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
            d3 = d3.append(d4,ignore_index=True)

        d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
        d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
        d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
        d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
        d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
        d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
        d3["VAR_NAME"] = "VAR"
        d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
        d3 = d3.replace([np.inf, -np.inf], 0)
        d3.IV = d3.IV.sum()

        return(d3)

    def char_bin(Y, X):

        df1 = pd.DataFrame({"X": X, "Y": Y})
        justmiss = df1[['X','Y']][df1.X.isnull()]
        notmiss = df1[['X','Y']][df1.X.notnull()]    
        df2 = notmiss.groupby('X',as_index=True)

        d3 = pd.DataFrame({},index=[])
        d3["COUNT"] = df2.count().Y
        d3["MIN_VALUE"] = df2.sum().Y.index
        d3["MAX_VALUE"] = d3["MIN_VALUE"]
        d3["EVENT"] = df2.sum().Y
        d3["NONEVENT"] = df2.count().Y - df2.sum().Y

        if len(justmiss.index) > 0:
            d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
            d4["MAX_VALUE"] = np.nan
            d4["COUNT"] = justmiss.count().Y
            d4["EVENT"] = justmiss.sum().Y
            d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
            d3 = d3.append(d4,ignore_index=True)

        d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
        d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
        d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
        d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
        d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
        d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
        d3["VAR_NAME"] = "VAR"
        d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
        d3 = d3.replace([np.inf, -np.inf], 0)
        d3.IV = d3.IV.sum()
        d3 = d3.reset_index(drop=True)

        return(d3)

    def data_vars(df1, target):

        stack = traceback.extract_stack()
        filename, lineno, function_name, code = stack[-2]
        vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
        final = (re.findall(r"[\w']+", vars_name))[-1]

        x = df1.dtypes.index
        count = -1

        for i in x:
            if i.upper() not in (final.upper()):
                if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                    conv = mono_bin(target, df1[i])
                    conv["VAR_NAME"] = i
                    count = count + 1
                else:
                    conv = char_bin(target, df1[i])
                    conv["VAR_NAME"] = i            
                    count = count + 1

                if count == 0:
                    iv_df = conv
                else:
                    iv_df = iv_df.append(conv,ignore_index=True)

        iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
        iv = iv.reset_index()
        return(iv_df,iv)
    df=df.fillna(df.mean())
    le = LabelEncoder()
    cat_cols=list(df.select_dtypes('object').columns)
    for fet in cat_cols:
        #le.fit(df[fet])
        df[fet] = le.fit_transform(df[fet].astype(str))
    target=input('ENTER THE TARGET VARIABLE : ')
    Target=df[target]
    final_iv, IV = data_vars(df,Target)
    final_iv
    final_iv.sort_values("VAR_NAME", inplace = True)
    a=final_iv.drop_duplicates(subset ="VAR_NAME")
    sorted_df=a.sort_values(by=['IV'],ascending=True)
    sorted_df
    c=sorted_df["IV"]
    threshold=float(input ('Enter threshold percentage to delete (in numerical form between 0.1 - 0.9 percent): '))
    b=list(np.where(c<threshold)[0])
    var=sorted_df['VAR_NAME']
    features=var.values[b].tolist()
    #d=var.loc[:b]
    #x=var.iloc[:b]
    #rejected_list=d.columns
    #features  ##THIS IS REJECTED LIST
    iv_values=sorted_df["IV"]
    data={'VAR_NAME':sorted_df["VAR_NAME"],'iV':sorted_df["IV"]}
    df1=pd.DataFrame(data)
    df1
    
    b=list(np.where(df1['iV']>threshold)[0])
    iv_values1=df1['iV'].values[b].tolist()  ###accepted iv values
    columns1=df1['VAR_NAME'].values[b].tolist()   ## accepted column names
    b1=list(np.where(df1['iV']<threshold)[0])
    iv_values2=df1['iV'].values[b1].tolist()  ###rejected iv values
    columns2=df1['VAR_NAME'].values[b1].tolist()   ## rejected column names
    def iv_plot(df):
        #b=list(np.where(a>threshold)[0])
        #x=df.iloc[:,b]
        #rejected_list=list(x.columns)
        trace = go.Bar(
                        x=iv_values2,
                        y=columns2,
                        marker=dict(color='#32E0C4'),orientation='h'
                        #marker_color=colors,
                        )
        data = [trace]

        layout = go.Layout(
                       barmode='overlay',
                       title="COUNT PLOT : {} ",
                       xaxis=dict(title="column names"),
                       yaxis=dict( title='Missing value percentage')
                       )
        fig = go.Figure(data=data, layout=layout)
        fig.add_trace(go.Bar(x=iv_values1,
                             y=columns1,
                                #base=0,
                                marker_color='lightslategrey',orientation='h'
                                #name='revenue'
                                ))
        fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)','paper_bgcolor' : 'rgba(0,0,0,0)'})
        fig.update_layout(title_text="<b>MISSING VALUE PERCENTAGE<b> ", title_x=0.5)
        fig.update_yaxes(title_text='<b>COLUMN NAMES<b>')
        fig.update_xaxes(title_text='<b>MISSING VALUE PERCENTAGE<b>')
        iplot(fig)
    return sorted_df,features,iv_plot(df)
    
   

In [54]:
IV(df)

ENTER THE TARGET VARIABLE : BAD
Enter threshold percentage to delete (in numerical form between 0.1 - 0.9 percent): 0.2


(   VAR_NAME    MIN_VALUE     MAX_VALUE  COUNT  EVENT  EVENT_RATE  NONEVENT  \
 0       BAD     0.000000      0.000000   4771      0    0.000000      4771   
 12   REASON     0.000000      1.000000   5708   1141    0.199895      4567   
 16      YOJ     0.000000      1.000000    849    183    0.215548       666   
 33  DEBTINC     0.524499      0.838118      3      2    0.666667         1   
 14      JOB     0.000000      1.000000   1715    304    0.177259      1411   
 32     CLNO     2.000000     71.000000   5892   1159    0.196707      4733   
 9     VALUE  8000.000000  73764.000000   1987    442    0.222446      1545   
 5   MORTDUE  2063.000000  48139.000000   1491    364    0.244131      1127   
 2      LOAN  1100.000000  12900.000000   2017    510    0.252851      1507   
 29     NINQ     0.000000      1.000000   3870    650    0.167959      3220   
 28    CLAGE   267.920922   1168.233561    852     85    0.099765       767   
 18    DEROG     0.000000      1.000000   5670   101